In [47]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

In [49]:
images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)

images_train = images[:20000]
images_test = imagesTest[-2000:]
labels_train = labels[:20000]
labels_test = labelsTest[-2000:]
images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/127.5)
images_test = images_test*(1/127.5)
images_train = images_train-1.0
images_test = images_test-1.0

# Should this be appended after or before normalization
images_train = np.insert(images_train, 0, 1, axis=1)
images_test = np.insert(images_test, 0, 1, axis=1)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10)

In [50]:
labelNums = 10

def labelsToVectForm(labels):
    vectForm = np.zeros((len(labels), labelNums))
    for i in range(len(labels)):
        vectForm[i][labels[i]] = 1
    return vectForm

t_train = labelsToVectForm(labels_train)
t_test = labelsToVectForm(labels_test)
t_validation = labelsToVectForm(labels_validation)

x = images_train
t = t_train

In [51]:
images_validation.shape

(2000, 785)

In [63]:
from sklearn.metrics import accuracy_score
import math

# Forward propagation

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmaxFunc(prod):
    prod = np.exp(prod)
    for i in range(len(prod)):
        rowSum = sum(prod[i, :])
        for j in range(len(prod[i, :])):
            prod[i][j] = prod[i][j]/rowSum    
    return prod

def getPredClass(y):
    predClass = np.argmax(y, axis=1)
    return predClass

def calculateOutput(x, W1, W2):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)
    X2 = np.insert(Z1, 0, 1, axis=1)
    A2 = X2.dot(W2)
    Y = softmaxFunc(A2)
    return Y

def calculateCost(x, t, W1, W2):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -1*(np.sum(np.multiply(t, np.log(Y))))
    return cost

def calculateCostOneInstance(x, t, W1, W2, n):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -1*(np.multiply(t, np.log(Y)))
    return np.sum(cost[n, :])

def calculateAccuracy(x, labels, W1, W2):
    Y = calculateOutput(x, W1, W2)
    predClass = getPredClass(Y)
    return accuracy_score(labels, predClass)


W1 = np.random.uniform(low=-1.0, high=1.0, size=(785, 64))
W2 = np.random.uniform(low=-1.0, high=1.0, size=(65, 10))
epsilon = 0.001
learningRate = 0.0001

for itr in range(5000):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)

#     print(Z1[0, :])
    X2 = np.insert(Z1, 0, 1, axis=1)
#     print(X2[0, :])
#     print("X2 : ")
#     print(X2)
#     print("W2 : ")
#     print(W2)
    A2 = X2.dot(W2)
#     print("A2 : ")
#     print(A2)
    Y = np.exp(A2)/np.sum(np.exp(A2), axis=1, keepdims = True)

#     Y_p = softmaxFunc(A2)
# #     print("Y-Y_p")
# #     print(Y-Y_p)

    # Backward propagation
    W2_nobias = np.delete(W2, (0), axis=0)
    summation_k = (Y-t).dot(W2_nobias.T)
    g_derivative = Z1*(1-Z1)
    delE_Wij = x.T.dot(np.multiply(summation_k, g_derivative))
    delE_Wij = delE_Wij
    
    # Check approximate derivative
#     for n in range(0, 150, 30):
#         print("For input to hidden layers")
#         for i in range(0, 785, 100):
#             for j in range(0, 64, 20):
#                 del_En_Wij = (g_derivative[n][j]*summation_k[n][j]*x[n][i])
#                 modifiedW1 = W1
#                 modifiedW1[i][j] += epsilon
#                 costFunction1 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
#                 modifiedW1[i][j] -= 2*epsilon
#                 costFunction2 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
#                 approx_delEn = (costFunction1-costFunction2)/(2*epsilon)
#                 diff = abs(del_En_Wij-approx_delEn)
# #                 print(del_En_Wij, " ", approx_delEn)
#                 if (diff > epsilon*epsilon):
#                     print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))
        
#         print("For hidden to output layers")
#         for j in range(0, 65, 20):
#             for k in range(0, 10, 3):
#                 actualDerivative = X2[n][j]*(Y[n][k]-t[n][k])
#                 modifiedW2 = W2
#                 modifiedW2[j][k] += epsilon
#                 costFunction1 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
#                 modifiedW2[j][k] -= 2*epsilon
#                 costFunction2 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
#                 approximateDerivative = (costFunction1-costFunction2)/(2*epsilon)
#                 diff = abs(actualDerivative-approximateDerivative)
# #                 print(actualDerivative, " ", approximateDerivative)
#                 if (diff > epsilon*epsilon):
#                     print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))
                
    W2 = W2-learningRate*((X2.T.dot(Y-t)))
    W1 = W1-learningRate*(delE_Wij)
    
    # Calculate validation error
    currTrainingError = calculateCost(x, t, W1, W2)
    currvalidationError = calculateCost(images_validation, t_validation, W1, W2)
    currTrainingAccuracy = calculateAccuracy(x, labels_train, W1, W2)
    currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1, W2)
    print("Current training error = ", currTrainingError)
    print("Current validation error = ", currvalidationError)
    print("Current training accuracy = ", currTrainingAccuracy)
    print("Current validation accuracy = ", currValidationAccuracy)
    

Current training error =  104576.8276189047
Current validation error =  11919.484318599905
Current training accuracy =  0.12033333333333333
Current validation accuracy =  0.12
Current training error =  88993.68009976037
Current validation error =  10165.253204233999
Current training accuracy =  0.18
Current validation accuracy =  0.1755
Current training error =  82035.54828948776
Current validation error =  9137.416381025134
Current training accuracy =  0.1947222222222222
Current validation accuracy =  0.2015
Current training error =  80396.71899177802
Current validation error =  8936.146426310566
Current training accuracy =  0.19944444444444445
Current validation accuracy =  0.192
Current training error =  73686.56286687643
Current validation error =  8027.648875656128
Current training accuracy =  0.22022222222222224
Current validation accuracy =  0.2255
Current training error =  84115.61898511862
Current validation error =  9363.46494418695
Current training accuracy =  0.208444444444

KeyboardInterrupt: 